Jupyter notebook extending SimPy tutorial found at

https://realpython.com/simpy-simulating-with-python/

# 1.0 Introduction

## 1.1 Credits, Sources, and Inspirations

Tutorial

https://realpython.com/simpy-simulating-with-python/

## 1.2 Imports and Installs

In [1]:
#import sys
#!pip install simpy

In [2]:
import simpy
import random
import statistics

import pandas as pd
import numpy

# 2.0 Classes and Functions

## 2.1 Theater Class

In [3]:
class Theater(object):
    def __init__(self, env, num_cashiers, num_servers, num_ushers):
        self.env = env
        self.cashier = simpy.Resource(env,num_cashiers)
        self.server = simpy.Resource(env, num_servers)
        self.usher = simpy.Resource(env, num_ushers)
        
    def purchase_ticket(self, moviegoer):
        yield self.env.timeout(random.randint(1,3))
        
    def check_ticket(self, moviegoer):
        #note that the units are 1 and it takes 3 seconds to check a ticket
        yield self.env.timeout(3 / 60)
        
    def sell_food(self, moviegoer):
        yield self.env.timeout(random.randint(1,5))

## 2.2 Moviegoer functions

In [4]:
def go_to_movies(env, moviegoer, theater):
    #moviegoer arrives at the theater
    arrival_time = env.now
    
    #using with tells simpy to release resource when done
    with theater.cashier.request() as request:
        yield request
        yield env.process(theater.purchase_ticket(moviegoer))
        
    with theater.usher.request() as request:
        yield request
        yield env.process(theater.check_ticket(moviegoer))
        
    if random.choice([True, False]):
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(moviegoer))
            
    #moviegoer goes into theater. wait time is over
    wait_times.append(env.now - arrival_time)

## 2.3 Run the theater process

In [5]:
def run_theater(env, num_cashier, num_servers, num_ushers):
    theater = Theater(env, num_cashier, num_servers, num_ushers)
    
    #expect 3 people in line at box office open
    for moviegoer in range(3):
        env.process(go_to_movies(env, moviegoer, theater))
        
    while True:
        yield env.timeout(0.20)  #wait 12 seconds
        
        moviegoer +=1
        env.process(go_to_movies(env, moviegoer, theater))
        

## 2.4 Measurement functions

In [6]:
def get_average_wait_time(wait_times):
    return statistics.mean(wait_times)
    

## 2.5 Simulation function

In [7]:
def run_sim(seed, num_cashiers, num_servers, num_ushers):
    #setup 
    random.seed(seed)  
    
    #initialize the simpy environment
    env = simpy.Environment()
    env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
    
    #run for 120 simulated minutes  (make a parameter later)
    env.run(until=90)
    
    print(wait_times)
    #return the results
    return get_average_wait_time(wait_times)

# 3.0 Run the simulation

At this level, we should explore the parameter space a bit. Since it is not a huge parameter space, some intelligently applied brute force should give us a lot of insight. To be "intelligent" the key is to think intuitively about the bottlenecks in the system.  Note individual process times:

- purchase between 1 and 3 minutes
- ticket validation 3 seconds 
- food selling .5 - 2.5 minutes 
- moviegoers arrive at the box office every 12 seconds (5 per minute)

In [8]:
sim_results = []
seed = 311  #i love douglas adams as much as the next nerd but...

max_cashiers = 1
max_servers = 1
max_users = 1

In [9]:
for cashiers in range(1,max_cashiers+1):
    for servers in range(1,max_servers+1):
        for ushers in range(1,max_users+1):
            
            #(re-)initialize wait_times
            #have to keep this as global for now
            wait_times = []
            
            avg_wait = run_sim(seed, cashiers, servers, ushers)
            this_result = (seed, cashiers, servers, ushers, avg_wait, len(wait_times))
            sim_results.append(this_result)
 

[6.05, 9.65, 11.05, 14.05, 13.25, 16.85, 16.85, 18.650000000000002, 21.45, 24.05, 27.45, 26.650000000000002, 30.249999999999996, 32.05, 36.849999999999994, 39.449999999999996, 40.24999999999999, 44.25, 49.05, 49.849999999999994, 48.24999999999999, 54.65, 55.64999999999999, 59.449999999999996, 62.05, 61.24999999999999, 62.05, 64.85, 65.64999999999999, 63.849999999999994, 67.44999999999999, 65.64999999999999, 69.05, 69.85, 72.44999999999999, 71.25, 72.44999999999999, 77.05, 78.44999999999999, 79.85, 80.25, 81.64999999999999, 81.05]


In [10]:
#dump results into a dataframe
cols = ['seed', 'cashiers', 'servers', 'ushers', 'avg_wait', 'num_customers']

sim_results_df = pd.DataFrame(sim_results, columns=cols)

sim_results_df.tail()

,seed,cashiers,servers,ushers,avg_wait,num_customers
0,311,1,1,1,48.654651,43


# 4.0 Analyze the results

In [11]:
sim_results_df.head()

,seed,cashiers,servers,ushers,avg_wait,num_customers
0,311,1,1,1,48.654651,43
